In [6]:
!pip install langchain_huggingface
!pip install -U langchain-community
!pip install faiss-cpu
!pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 29.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [25]:
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from underthesea import word_tokenize
import faiss
import numpy as np
import torch
import re
import string

In [26]:
def load_qa_data(questions_file, answers_file):
    with open(questions_file, 'r', encoding='utf-8') as q_file, open(answers_file, 'r', encoding='utf-8') as a_file:
        questions = q = q_file.readlines()
        answers = a_file.readlines()
        qa_pairs = [{"question": q.strip(), "answer": a.strip()} for q, a in zip(questions, answers)]
    return qa_pairs

train_data = load_qa_data('data/train/questions.txt', 'data/train/reference_answers.txt')
test_data = load_qa_data('data/test/questions.txt', 'data/test/reference_answers.txt')
# qa_data = load_qa_data('data/questions_answers/questions/questions.txt', 'data/questions_answers/answers/reference_answers.txt')

In [27]:
train_data

[{'question': 'Tên của trường đại học đầu tiên ở Việt Nam là gì?',
  'answer': 'đại học'},
 {'question': 'Trọng tâm chính của trường mới thành lập tại Đại học Quốc gia Việt Nam là gì?',
  'answer': 'dược'},
 {'question': 'Tên của trường đại học mới thành lập năm 2003 là gì?',
  'answer': 'đã'},
 {'question': 'Chính phủ đã làm gì với Khoa Giáo dục?', 'answer': 'nâng cấp'},
 {'question': 'Mục đích của việc thành lập Đại học Quốc gia là gì?',
  'answer': 'với'},
 {'question': 'Đại học Quốc gia Việt Nam giống với kiểu nền văn minh nào?',
  'answer': 'con người'},
 {'question': 'Tên gọi của đợt sáp nhập và tổ chức lại Trung tâm Ứng dụng công nghệ thông tin là gì?',
  'answer': 'dựa trên'},
 {'question': 'Tên của điều lệ do Thủ tướng Chính phủ ban hành năm 2001 là gì?',
  'answer': 'hoạt động'},
 {'question': 'Tên gọi mới của Trung tâm Nghiên cứu Phát triển và Đảm bảo chất lượng giáo dục là gì?',
  'answer': 'giáo dục'},
 {'question': 'Đại học Quốc gia chính thức đi vào hoạt động khi nào?',


In [28]:
test_data

[{'question': 'Thủ tướng Chính phủ đã ban hành điều gì vào năm 2002?',
  'answer': 'quyết định'},
 {'question': 'Tên của trường đại học được tách ra từ Trường Đại học Khoa học Tự nhiên là gì?',
  'answer': 'giáo dục'},
 {'question': 'Thủ tướng Chính phủ đã làm gì vào năm 2004?',
  'answer': 'tham dự'},
 {'question': 'Cơ sở cho việc thành lập Trường Đại học Công nghệ là gì?',
  'answer': 'cơ sở'},
 {'question': 'Các đơn vị khác nằm trong Đại học Quốc gia Hà Nội là gì?',
  'answer': 'khác'},
 {'question': 'Một số đơn vị trực thuộc Đại học Quốc gia Hà Nội sẽ được chuyển đến đâu vào năm 2015?',
  'answer': 'khuôn viên'},
 {'question': 'Các đơn vị khác nằm trong Đại học Quốc gia Hà Nội là gì?',
  'answer': 'khác'},
 {'question': 'Tên của hệ thống đại học ở Việt Nam là gì?',
  'answer': 'quốc gia'},
 {'question': 'Tên của Đại học Quốc gia Hà Nội là gì?', 'answer': 'đại học'},
 {'question': 'Quốc gia nào có khu đô thị đại học lớn nhất Việt Nam?',
  'answer': 'việt nam'},
 {'question': 'Loại c

In [29]:
# Tạo embedding
embedder = SentenceTransformer('intfloat/multilingual-e5-large')
train_questions = [qa['question'] for qa in train_data]
question_embeddings = embedder.encode(train_questions, convert_to_numpy=True)

# Tạo index FAISS
dimension = question_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(question_embeddings)

In [30]:
# Tạo vector store
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')
vector_store = FAISS.from_texts(train_questions, embeddings, metadatas=train_data)

# Load mô hình và tokenizer cho Qwen2-0.5B-Instruct
model_id = "Qwen/Qwen2-0.5B-Instruct"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",  # Chạy trên CPU
        torch_dtype=torch.float16
    )
except Exception as e:
    print(f"Lỗi khi tải mô hình: {e}")
    exit()

# Tạo pipeline text-generation
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
)

# Tạo HuggingFacePipeline cho LangChain
llm = HuggingFacePipeline(pipeline=pipe)

# Tùy chỉnh prompt
prompt_template = """Dựa trên thông tin sau, trả lời câu hỏi một cách chính xác, ngắn gọn và bằng tiếng Việt:

Thông tin: {context}

Câu hỏi: {question}

Câu trả lời: """
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Tạo RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

Device set to use cuda:0


In [32]:
# Hàm trích xuất câu trả lời cuối cùng
def extract_answer(output):
    # Tách output thành các dòng
    lines = output.strip().split('\n')
    # Tìm dòng cuối cùng bắt đầu bằng "Câu trả lời: "
    for line in reversed(lines):
        if line.startswith("Câu trả lời: "):
            return line[len("Câu trả lời: "):].strip()
    return output.strip()  # Nếu không tìm thấy, trả về output đã làm sạch

In [31]:
# evaluation
# Hàm chuẩn hóa và đánh giá
def normalize_text(text):
    text = text.lower()
    text = re.sub(f'[{string.punctuation}]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def tokenize(text):
    return word_tokenize(text, format="text").split()

def compute_metrics(system_answer, reference_answer):
    system_tokens = tokenize(normalize_text(system_answer))
    reference_tokens = tokenize(normalize_text(reference_answer))
    em = 1 if normalize_text(system_answer) == normalize_text(reference_answer) else 0
    common_tokens = set(system_tokens) & set(reference_tokens)
    recall = len(common_tokens) / len(reference_tokens) if reference_tokens else 0
    precision = len(common_tokens) / len(system_tokens) if system_tokens else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return {"exact_match": em, "recall": recall, "precision": precision, "f1": f1}

In [33]:
# Đánh giá trên dataset test
total_metrics = {"exact_match": 0, "recall": 0, "precision": 0, "f1": 0}
num_questions = len(test_data)

for item in test_data:
    try:
        result = qa_chain.invoke({"query": item['question']})
        system_answer = extract_answer(result['result'])
        reference_answer = item['answer']
        metrics = compute_metrics(system_answer, reference_answer)
        for key in total_metrics:
            total_metrics[key] += metrics[key]
    except Exception as e:
        print(f"Lỗi khi xử lý câu hỏi '{item['question']}': {e}")

# Tính trung bình
for key in total_metrics:
    total_metrics[key] /= num_questions if num_questions > 0 else 1

# In hiệu suất
print("\nHiệu suất trung bình trên dataset test:")
print(f"Exact Match: {total_metrics['exact_match']:.4f}")
print(f"Recall: {total_metrics['recall']:.4f}")
print(f"Precision: {total_metrics['precision']:.4f}")
print(f"F1: {total_metrics['f1']:.4f}")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Hiệu suất trung bình trên dataset test:
Exact Match: 0.0000
Recall: 0.1486
Precision: 0.0191
F1: 0.0318


In [22]:
# Vòng lặp để nhập câu hỏi
while True:
    query = input("Nhập câu hỏi của bạn (hoặc nhấn Enter để thoát): ")
    if not query.strip():
        print("Đã thoát.")
        break
    try:
        result = qa_chain.invoke({"query": query})
        answer = extract_answer(result['result'])  # Trích xuất câu trả lời
        print(answer)
    except Exception as e:
        print(f"Lỗi khi xử lý câu hỏi: {e}")

Nhập câu hỏi của bạn (hoặc nhấn Enter để thoát): Tên của trường đại học đầu tiên ở Việt Nam là gì?
Trường Đại học Quốc gia Việt Nam.
Nhập câu hỏi của bạn (hoặc nhấn Enter để thoát): Tên của trường đại học mới thành lập năm 2003 là gì?
Trường Đại học Khoa học Tự nhiên.
Nhập câu hỏi của bạn (hoặc nhấn Enter để thoát): trường đại học nào thành lập vào năm 2003
Trường Đại học Giáo dục.
Nhập câu hỏi của bạn (hoặc nhấn Enter để thoát): trường đại học quốc tử giám là trường nào
Trường Đại học Quốc Gia Hà Nội là trường đại học.
Nhập câu hỏi của bạn (hoặc nhấn Enter để thoát): Trường Đại học Quốc gia Việt Nam thành lập vào năm nào
1958.
Nhập câu hỏi của bạn (hoặc nhấn Enter để thoát): 
Đã thoát.
